# Secure Federated Linear Regression – Single‑Round Evaluation
*(Protocol v3 – Closed‑Form)*

This notebook benchmarks **Protocol v3** on **five datasets** (Synthetic, California Housing, Diabetes, Ames Housing, Wine Quality Red).

For each dataset we compare five approaches:

- **Centralized** (pooled data)  
- **Naïve Federated** (no privacy) – same as HE-FL but without encryption.
- **Differential-Privacy FL** (Gaussian noise, σ = 10)  
- **Secure Protocol v3** (random β ∈ [−0.1, 0.1])  

Metrics reported: **MSE**, **MAE**, **R²**.


## 1. Imports

In [1]:
import numpy as np, pandas as pd
from sklearn.datasets import fetch_california_housing, load_diabetes, fetch_openml
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# warnings.filterwarnings('ignore')

In [2]:
SEED = 43
N_CLIENTS = 10
np.random.seed(SEED)

## 2. Helper functions

In [3]:
def add_intercept(X):
    return np.hstack([np.ones((X.shape[0], 1)), X])

def split_random(X, y, seed=SEED, test_size=0.1):
    rng = np.random.RandomState(seed)

    idx = rng.permutation(len(y))
    split = int((1 - test_size) * len(y))

    return X[idx[:split]], X[idx[split:]], y[idx[:split]], y[idx[split:]]

def make_random_clients(X, y, n=10, seed=SEED):
    idx = np.random.RandomState(seed).permutation(len(y))
    chunks = np.array_split(idx, n)

    clients=[]
    for i, ch in enumerate(chunks):
        X_train, X_test, y_train, y_test = split_random(X[ch], y[ch], seed=i)
        clients.append({
            'X_train': add_intercept(X_train),
            'y_train': y_train,
            'X_test': add_intercept(X_test),
            'y_test': y_test
        })
    return clients


### Dataset loaders

In [4]:
def clients_synth(n_clients: int = N_CLIENTS,
                  n_features: int = 7,
                  min_size: int = 1000,
                  max_size: int = 10000,
                  missing_feature_prob: float = 0.25,
                  seed: int = SEED):
    """Return a list of dicts {'X_train','y_train','X_test','y_test'}"""
    rng = np.random.default_rng(seed)
    clients = []

    DATA_SIZE = rng.integers(min_size, max_size, size=n_clients)
    MEAN = np.random.randint(0, 20, size=n_clients)
    STD = np.random.randint(5, 10, size=n_clients)
    NOISE_STD = np.array([np.random.uniform(0, 0.5*std) for std in STD])          # low
    # NOISE_STD = np.array([np.random.uniform(0.5*std, std) for std in STD])      # medium
    # NOISE_STD = np.array([np.random.uniform(std, 2*std) for std in STD])        # high

    for i in range(n_clients):
        X = rng.normal(MEAN[i], STD[i], size=(DATA_SIZE[i], n_features))

        # Global ground‑truth weight vector (for all clients) – random for each experiment
        w = rng.normal(0, 1, n_features)

        y = X @ w + rng.normal(0, 1, DATA_SIZE[i])

        # Add Gaussian noise to the target values
        y += rng.normal(0, NOISE_STD[i], DATA_SIZE[i])

        Xtr, Xte, ytr, yte = split_random(X, y, seed=i)
        clients.append(
            {
                "X_train": add_intercept(Xtr),
                "y_train": ytr,
                "X_test": add_intercept(Xte),
                "y_test": yte,
            }
        )
    return clients


def clients_cal(seed=SEED):
    d = fetch_california_housing()
    X, y = d.data, d.target

    latitudes = X[:, 6]
    slices = np.array_split(np.argsort(latitudes), N_CLIENTS)      # split equally for 10 clients

    cl = []
    for i, sl in enumerate(slices):
        Xtr, Xte, ytr, yte = split_random(X[sl], y[sl], seed=i)
        cl.append(
            {
                "X_train": add_intercept(Xtr),
                "y_train": ytr,
                "X_test": add_intercept(Xte),
                "y_test": yte,
            }
        )
    return cl


def clients_diab(seed=SEED):
    d = load_diabetes()
    return make_random_clients(d.data, d.target, 10, seed)


def clients_ames(seed=SEED):
    df = fetch_openml("house_prices", as_frame=True, parser="pandas").frame
    df = df.select_dtypes(include=["number"]).dropna()

    y = df["SalePrice"].astype(float).values
    X = df.drop(columns=["SalePrice"]).to_numpy(dtype=float)

    return make_random_clients(X, y, 10, seed)


def clients_wine(seed=SEED):
    df = fetch_openml("wine-quality-red", as_frame=True, parser="pandas").frame

    X = df.drop(columns=["class"]).astype(float).to_numpy()
    y = df["class"].astype(float).to_numpy()

    return make_random_clients(X, y, 10, seed)


### Training algorithms

In [5]:
def central(clients):
    X = np.vstack([c["X_train"] for c in clients])
    y = np.hstack([c["y_train"] for c in clients])

    return np.linalg.solve(X.T @ X, X.T @ y)


def naive(clients):
    d = clients[0]["X_train"].shape[1]
    XtX = np.zeros((d, d))
    Xty = np.zeros(d)

    for c in clients:
        XtX += c["X_train"].T @ c["X_train"]
        Xty += c["X_train"].T @ c["y_train"]

    return np.linalg.solve(XtX, Xty)


def dp(clients, sigma=10, seed=SEED):
    rng = np.random.default_rng(seed)

    d = clients[0]["X_train"].shape[1]
    XtX = np.zeros((d, d))
    Xty = np.zeros(d)

    for c in clients:
        XtX += c["X_train"].T @ c["X_train"] + rng.normal(scale=sigma, size=(d, d))
        Xty += c["X_train"].T @ c["y_train"] + rng.normal(scale=sigma, size=d)

    return np.linalg.solve(XtX, Xty)


def secure(clients, beta=(-0.1, 0.1), seed=SEED):
    rng = np.random.default_rng(seed)

    n = clients[0]["X_train"].shape[1] - 1

    while True:
        W = rng.normal(size=(n + 2, n + 2))
        W[-1, :] = 1.0
        if np.linalg.matrix_rank(W) == n + 2:
            break

    L = np.zeros((n + 1, n + 2))

    for c in clients:
        X, y = c["X_train"], c["y_train"]
        Y = np.tile(y.reshape(-1, 1), (1, n + 2))

        Li = X.T @ (X @ W[:-1, :] - Y)

        beta_val = rng.uniform(*beta)

        L += (1 - beta_val) * Li

    C = L @ np.linalg.inv(W)
    A, b = C[:, :-1], C[:, -1]

    return -np.linalg.solve(A, b)


def evaluate(w, clients):
    X = np.vstack([c["X_test"] for c in clients])
    y = np.hstack([c["y_test"] for c in clients])

    return (
        mean_squared_error(y, X @ w),
        mean_absolute_error(y, X @ w),
        r2_score(y, X @ w),
    )


## 3. Run benchmarks

In [6]:
datasets = {
    "Synthetic": clients_synth(),
    "California Housing": clients_cal(),
    "Diabetes": clients_diab(),
    "Ames Housing": clients_ames(),
    "Wine Quality": clients_wine(),
}

tables = {}

for name, cl in datasets.items():
    metrics = {
        "Centralized": evaluate(central(cl), cl),
        "Naïve FL": evaluate(naive(cl), cl),
        "DP σ=10": evaluate(dp(cl), cl),
        "Protocol v2": evaluate(secure(cl, beta=(-0.05, 0.05),), cl),
    }

    tables[name] = pd.DataFrame(metrics, index=["MSE", "MAE", "R2"]).T


In [8]:
for name,df in tables.items():
    print('\n', name)
    display(df.style.format({
        'MSE': '{:.4f}',
        'MAE': '{:.4f}',
        'R2': '{:.4f}'
        })
    )


 Synthetic


,MSE,MAE,R2
Centralized,615.4153,17.3641,0.2376
Naïve FL,615.4153,17.3641,0.2376
DP σ=10,615.4151,17.3641,0.2376
Protocol v3,615.3715,17.3625,0.2377



 California Housing


,MSE,MAE,R2
Centralized,0.5511,0.5482,0.6037
Naïve FL,0.5511,0.5482,0.6037
DP σ=10,0.6554,0.5991,0.5287
Protocol v3,0.5510,0.5483,0.6037



 Diabetes


,MSE,MAE,R2
Centralized,2828.6861,43.7746,0.5936
Naïve FL,2828.6861,43.7746,0.5936
DP σ=10,8735.3364,83.7024,-0.2550
Protocol v3,2824.0446,43.7125,0.5943



 Ames Housing


,MSE,MAE,R2
Centralized,2206027601.3236,26479.2917,0.6531
Naïve FL,2206027601.3236,26479.2917,0.6531
DP σ=10,2275029826.1510,28668.0888,0.6422
Protocol v3,2212885168.7002,26507.0581,0.6520



 Wine Quality


,MSE,MAE,R2
Centralized,0.3893,0.4677,0.3441
Naïve FL,0.3893,0.4677,0.3441
DP σ=10,0.7639,0.6752,-0.2869
Protocol v3,0.3892,0.4676,0.3442
